In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
data = requests.get('https://api.covid19india.org/v4/data-all.json').json()

In [ ]:
for date in data.keys():
    del data[date]['TT']
    print(date)
    for state in data[date].keys():
        if 'districts' in data[date][state].keys():
            for key in data[date][state]['districts'].keys():
                try:
                    data[date][state]['districts'][key] = data[date][state]['districts'][key]['total']
                except Exception as e:
                    print(data[date][state]['districts'][key])

In [ ]:
df_districts_all = pd.DataFrame(columns=['date', 'state', 'district', 'confirmed', 'active', 'recovered', 'deceased', 'tested', 'migrated'])
for date in data.keys():
    if date > '2020-04-30':
        for state in data[date].keys():
            try:
                df_date_state = pd.DataFrame.from_dict(data[date][state]['districts']).T.reset_index()
            except Exception as e:
                print(e)
                continue
            df_date_state = df_date_state.rename({'index' : 'date'}, axis='columns')
            df_date_state['active'] = df_date_state['confirmed'] - (df_date_state['recovered'] + df_date_state['deceased'])
            df_date_state['state'] = statecode_to_state_dict(state)
            df_date_state['date'] = date
            df_districts_all = pd.concat([df_districts_all, df_date_state], ignore_index=True)